In [1]:
!python -m pip install cytetype

ERROR: Ignored the following versions that require a different python version: 0.1.1 Requires-Python >=3.12; 0.1.2 Requires-Python >=3.12; 0.1.3 Requires-Python >=3.12; 0.1.4 Requires-Python >=3.12; 0.2.0 Requires-Python >=3.12; 0.2.1 Requires-Python >=3.12; 0.3.0 Requires-Python >=3.12; 0.3.1 Requires-Python >=3.12; 0.3.2 Requires-Python >=3.12; 0.3.3 Requires-Python >=3.12; 0.3.4 Requires-Python >=3.12; 0.3.5 Requires-Python >=3.12
ERROR: Could not find a version that satisfies the requirement cytetype (from versions: none)
ERROR: No matching distribution found for cytetype
